<a href="https://colab.research.google.com/github/Flamingo23456/AI_bootcamp/blob/main/cifar100_cnn_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import dependencies
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms
from torch.utils.data import DataLoader


In [2]:
#transforms
transform=transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5),(0.5))
                              ])


In [3]:
#load data
train_data=datasets.FashionMNIST(root='./dir',
                             train=True,
                             download=True,
                             transform=transform)
test_data=datasets.FashionMNIST(root='./dir',
                            train=False,
                            download=True,
                            transform=transform)

train_loader=DataLoader(train_data,
                        batch_size=128,
                        shuffle=True)
test_loader=DataLoader(test_data,
                       batch_size=128,
                       shuffle=False)


100%|██████████| 26.4M/26.4M [00:02<00:00, 8.82MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 152kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.75MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 14.5MB/s]


In [4]:
#architecture
class cnn_fashionmnist(nn.Module):
  def __init__(self):
    super(cnn_fashionmnist,self).__init__()
    self.conv1=nn.Conv2d(1,32,3,padding=1)#convolution layer
    self.pool1=nn.MaxPool2d(2,2)#pooling layer

    self.conv2=nn.Conv2d(32,64,3)#convolution layer
    self.pool2=nn.MaxPool2d(2,2)#pooling layer

    #FahionMNIST size is 28X28X1
    #Converted to 14X14X1
    #Converted to 14X14X64
    #Then to 7X7X64
    #Dense layers
    self.fc1=nn.Linear(6*6*64,256)
    self.fc2=nn.Linear(256,128)
    self.fc3=nn.Linear(128,10)

  def forward(self,x):
    #x represents the image input
    x=self.conv1(x)
    x=torch.relu(x)
    x=self.pool1(x)

    x=self.pool2(torch.relu(self.conv2(x)))
    #x=x.Flatten(1)#x=x.view(x.size(0),-1)
    x=x.view(x.size(0),-1) # Flatten the tensor
    x=self.fc1(x)
    x=torch.relu(x)
    x=self.fc2(x)
    x=torch.relu(x)
    x=self.fc3(x)
    return x

In [5]:
#object creation
model=cnn_fashionmnist()
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),
                     lr=0.001)

In [10]:
#train
for epoch in range(5):
  for image,label in train_loader:
    output=model(image)
    loss=criterion(output,label)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
  print(f"epoch:{epoch+1},loss:{loss}")

epoch:1,loss:0.11279643326997757
epoch:2,loss:0.09355369955301285
epoch:3,loss:0.04243222251534462
epoch:4,loss:0.03865954652428627
epoch:5,loss:0.03928624466061592


In [11]:
correct=0.0
total=0.0
model.eval()
with torch.no_grad():
  output=model(image)
  max,predicted=torch.max(output,1)
  correct+=(predicted==label).sum().item()
  total+=label.size(0)
print(f"test accuracy:{(correct/total)*100}")

test accuracy:100.0
